# Strategy analysis example

Debugging a strategy can be time-consuming. FreqTrade offers helper functions to visualize raw data.

## Setup

In [1]:
import os
from pathlib import Path
# Change directory
# Modify this cell to insure that the output shows the correct path.
# Define all paths relative to the project root shown in the cell output
project_root = "/home/cryo/freqtrade"
i=0
try:
    os.chdirdir(project_root)
    assert Path('LICENSE').is_file()
except:
    while i<4 and (not Path('LICENSE').is_file()):
        os.chdir(Path(Path.cwd(), '../'))
        i+=1
    project_root = Path.cwd()
print(Path.cwd())

# Customize these according to your needs.

# Define some constants
timeframe = "5m"
# Name of the strategy class
strategy_name = 'SampleStrategy'
# Path to user data
user_data_dir = Path('user_data')
# Location of the strategy
strategy_location = user_data_dir / 'strategies'
# Location of the data
data_location = Path(user_data_dir, 'data', 'binance')
# Pair to analyze - Only use one pair here
pair = "BTC_USDT"

/home/cryo/freqtrade


In [2]:
# Load data using values set above
from freqtrade.data.history import load_pair_history

candles = load_pair_history(datadir=data_location,
                            timeframe=timeframe,
                            pair=pair)

# Confirm success
print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {data_location}")
candles.head()

Loaded 8639 rows of data for BTC_USDT from user_data/data/binance


,date,open,high,low,close,volume
0,2019-10-15 21:55:00+00:00,8146.95,8159.00,8144.49,8157.99,76.948556
1,2019-10-15 22:00:00+00:00,8157.99,8158.19,8150.08,8156.38,44.493666
2,2019-10-15 22:05:00+00:00,8155.70,8157.57,8150.26,8150.93,43.126204
3,2019-10-15 22:10:00+00:00,8150.96,8157.00,8150.08,8155.20,36.664101
4,2019-10-15 22:15:00+00:00,8155.08,8157.91,8151.22,8157.46,50.256138


## Load and run strategy
* Rerun each time the strategy file is changed

In [3]:
# Load strategy using values set above
from freqtrade.resolvers import StrategyResolver
strategy = StrategyResolver.load_strategy({'strategy': strategy_name,
                                          'user_data_dir': user_data_dir,
                                          'strategy_path': strategy_location})

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {'pair': pair})
df.tail()

,date,open,high,low,close,volume,rsi,adx,macd,macdsignal,...,fastk,bb_lowerband,bb_middleband,bb_upperband,sar,tema,htsine,htleadsine,buy,sell
8634,2019-11-14 21:25:00+00:00,8663.28,8668.22,8662.00,8667.83,29.133722,59.280830,18.031568,3.823066,3.665416,...,59.455873,8647.935994,8660.753000,8673.570006,8673.468080,8665.297896,-0.474944,-0.958101,NaN,NaN
8635,2019-11-14 21:30:00+00:00,8667.42,8673.00,8664.89,8669.73,41.453415,60.872243,18.585650,4.062275,3.744787,...,76.857749,8650.631424,8661.916833,8673.202243,8673.176118,8667.472412,-0.499157,-0.965674,NaN,NaN
8636,2019-11-14 21:35:00+00:00,8669.65,8672.87,8666.12,8670.94,26.497043,61.893648,19.100156,4.299920,3.855814,...,85.421090,8652.616525,8662.922333,8673.228142,8673.000000,8669.400051,-0.516652,-0.970750,NaN,NaN
8637,2019-11-14 21:40:00+00:00,8672.18,8673.00,8665.00,8666.09,52.826155,55.625648,19.307801,4.050212,3.894694,...,50.642857,8653.585802,8663.539500,8673.493198,8658.870000,8668.181694,-0.518600,-0.971294,NaN,NaN
8638,2019-11-14 21:45:00+00:00,8665.73,8667.50,8662.05,8665.80,45.890402,55.265256,18.815865,3.785282,3.872811,...,34.545455,8654.136224,8663.855167,8673.574110,8659.152600,8667.183442,-0.515584,-0.970450,NaN,NaN


### Display the trade details

* Note that using `data.head()` would also work, however most indicators have some "startup" data at the top of the dataframe.
* Some possible problems
    * Columns with NaN values at the end of the dataframe
    * Columns used in `crossed*()` functions with completely different units
* Comparison with full backtest
    * having 200 buy signals as output for one pair from `analyze_ticker()` does not necessarily mean that 200 trades will be made during backtesting.
    * Assuming you use only one condition such as, `df['rsi'] < 30` as buy condition, this will generate multiple "buy" signals for each pair in sequence (until rsi returns > 29). The bot will only buy on the first of these signals (and also only if a trade-slot ("max_open_trades") is still available), or on one of the middle signals, as soon as a "slot" becomes available.  


In [4]:
# Report results
print(f"Generated {df['buy'].sum()} buy signals")
data = df.set_index('date', drop=False)
data.tail()

Generated 54.0 buy signals


,date,open,high,low,close,volume,rsi,adx,macd,macdsignal,...,fastk,bb_lowerband,bb_middleband,bb_upperband,sar,tema,htsine,htleadsine,buy,sell
date,,,,,,,,,,,,,,,,,,,,,
2019-11-14 21:25:00+00:00,2019-11-14 21:25:00+00:00,8663.28,8668.22,8662.00,8667.83,29.133722,59.280830,18.031568,3.823066,3.665416,...,59.455873,8647.935994,8660.753000,8673.570006,8673.468080,8665.297896,-0.474944,-0.958101,NaN,NaN
2019-11-14 21:30:00+00:00,2019-11-14 21:30:00+00:00,8667.42,8673.00,8664.89,8669.73,41.453415,60.872243,18.585650,4.062275,3.744787,...,76.857749,8650.631424,8661.916833,8673.202243,8673.176118,8667.472412,-0.499157,-0.965674,NaN,NaN
2019-11-14 21:35:00+00:00,2019-11-14 21:35:00+00:00,8669.65,8672.87,8666.12,8670.94,26.497043,61.893648,19.100156,4.299920,3.855814,...,85.421090,8652.616525,8662.922333,8673.228142,8673.000000,8669.400051,-0.516652,-0.970750,NaN,NaN
2019-11-14 21:40:00+00:00,2019-11-14 21:40:00+00:00,8672.18,8673.00,8665.00,8666.09,52.826155,55.625648,19.307801,4.050212,3.894694,...,50.642857,8653.585802,8663.539500,8673.493198,8658.870000,8668.181694,-0.518600,-0.971294,NaN,NaN
2019-11-14 21:45:00+00:00,2019-11-14 21:45:00+00:00,8665.73,8667.50,8662.05,8665.80,45.890402,55.265256,18.815865,3.785282,3.872811,...,34.545455,8654.136224,8663.855167,8673.574110,8659.152600,8667.183442,-0.515584,-0.970450,NaN,NaN


## Load existing objects into a Jupyter notebook

The following cells assume that you have already generated data using the cli.  
They will allow you to drill deeper into your results, and perform analysis which otherwise would make the output very difficult to digest due to information overload.

### Load backtest results to pandas dataframe

Analyze a trades dataframe (also used below for plotting)

In [5]:
from freqtrade.data.btanalysis import load_backtest_data

# Load backtest results
trades = load_backtest_data(user_data_dir / "backtest_results/backtest-result.json")

# Show value-counts per pair
trades.groupby("pair")["sell_reason"].value_counts()

pair      sell_reason
BTC/USDT  roi            17
          force_sell      1
Name: sell_reason, dtype: int64

### Load live trading results into a pandas dataframe

In case you did already some trading and want to analyze your performance

In [6]:
from freqtrade.data.btanalysis import load_trades_from_db

# Fetch trades from database
trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
trades.groupby("pair")["sell_reason"].value_counts()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 0 but corresponding boolean dimension is 1

## Analyze the loaded trades for trade parallelism
This can be useful to find the best `max_open_trades` parameter, when used with backtesting in conjunction with `--disable-max-market-positions`.

`analyze_trade_parallelism()` returns a timeseries dataframe with an "open_trades" column, specifying the number of open trades for each candle.

In [7]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, '5m')


parallel_trades.plot()

ValueError: No objects to concatenate

## Plot results

Freqtrade offers interactive plotting capabilities based on plotly.

In [ ]:
from freqtrade.plot.plotting import  generate_candlestick_graph
# Limit graph period to keep plotly quick and reactive

data_red = data['2019-06-01':'2019-06-10']
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades,
                                   indicators1=['sma20', 'ema50', 'ema55'],
                                   indicators2=['rsi', 'macd', 'macdsignal', 'macdhist']
                                  )




In [ ]:
# Show graph inline
# graph.show()

# Render graph in a seperate window
graph.show(renderer="browser")


Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.